# 01_preprocessing.ipynb

## Objective

Preprocess Cusanus' sermons for topic modeling:

- Load and parse TEI XML files.
- Clean, normalize, and lemmatize text.
- Save cleaned text for further analysis.


In [1]:
import os
from bs4 import BeautifulSoup
import tqdm
from collections import defaultdict
import yaml

In [2]:
# 更改工作目录到项目根目录
os.chdir('/Users/jessie/Documents/Projects/Cusanus_Topic_Modeling')
print("当前工作目录为: ", os.getcwd())


当前工作目录为:  /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling


In [3]:
# 加载配置文件
CONFIG_PATH = 'config.yaml'

try:
    with open(CONFIG_PATH, 'r') as config_file:
        config = yaml.safe_load(config_file)
    BASE_DIR = config.get('BASE_DIR', os.getcwd())
    print("配置文件加载成功，项目根目录为: ", BASE_DIR)
except FileNotFoundError:
    print(f"配置文件未找到: {CONFIG_PATH}，使用当前工作目录。")
    BASE_DIR = os.getcwd()

# 示例使用 BASE_DIR
input_dir = os.path.join(BASE_DIR, 'data/raw')
output_dir = os.path.join(BASE_DIR, 'data/processed')

print(f"输入目录: {input_dir}")
print(f"输出目录: {output_dir}")

# 列出输入目录中的文件
print("输入目录中的文件: ", os.listdir(input_dir))

配置文件加载成功，项目根目录为:  /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling
输入目录: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/raw
输出目录: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed
输入目录中的文件:  ['v170_048.xml', 'v170_060.xml', 'v170_074.xml', 'h180_134.xml', 'v180_135.xml', 'h170_075.xml', 'h170_061.xml', 'h170_049.xml', 'h190_281.xml', 'v160_024_1.xml', 'h190_256.xml', 'h170_101.xml', 'h170_115.xml', 'h190_242.xml', 'v190_243.xml', 'v170_114.xml', 'v170_100.xml', 'v190_257.xml', 'v190_280.xml', 'v190_281.xml', 'v190_256.xml', 'v170_101.xml', 'v170_115.xml', 'v190_242.xml', 'h190_243.xml', 'h170_114.xml', 'h170_100.xml', 'h190_257.xml', 'h190_280.xml', 'h170_048.xml', 'h170_060.xml', 'h170_074.xml', 'v180_134.xml', 'h180_135.xml', 'v170_075.xml', 'v170_061.xml', 'v170_049.xml', 'v170_077.xml', 'v170_063.xml', 'h180_123.xml', 'v170_088.xml', 'h180_137.xml', 'v180_136.xml', 'v180_122.xml', 'h170_089.xml', 'h170_062.xml', 'h170_076.xml', 'h190_282.xm

# 数据预处理部分

In [4]:
# 停用词加载函数，过滤掉空行和注释行
def load_stopwords(filepath):
    try:
        with open(filepath, 'r') as f:
            stopwords = {line.strip() for line in f if line.strip() and not line.startswith('#')}
        print("停用词列表加载成功。")
        return stopwords
    except FileNotFoundError:
        print(f"停用词文件未找到: {filepath}")
        return set()

# 加载词形表和停用词
stopwords_path = os.path.join(BASE_DIR, config.get('STOPWORDS_PATH', 'data/external/stopwords_latin.txt'))
latin_stopwords = load_stopwords(stopwords_path)

停用词列表加载成功。


In [13]:
# 词形表加载函数
# 从 lemma.xml 文件中加载词形，还原词汇的标准形式
# 解析 lemma.xml 文件并创建 lemma_dict
def load_lemma_dict(lemma_file):
    with open(lemma_file, 'r') as file:
        soup = BeautifulSoup(file, 'xml')
        lemma_dict = {}
        for lemma in soup.find_all('lemma'):
            id_lemma = lemma.get('id_lemma')
            name = lemma.get('name')
            if id_lemma and name:
                lemma_dict[int(id_lemma)] = name
        return lemma_dict


In [14]:
# 加载词典
lemma_dict = load_lemma_dict("data/raw/lemma.xml")

In [15]:
# XML 文件预处理函数
# 处理 XML 文件，检查是否包含 lemma_l 属性，并根据情况进行词形还原和停用词去除
def preprocess_text(xml_file):
    try:
        with open(xml_file, 'r') as file:
            soup = BeautifulSoup(file, 'xml')
            words = []

            for w in soup.find_all('w'):
                lemma_attr = w.get('lemma_l')
                
                # 若 lemma_l 存在且为数字，用 lemma_dict 词典获取还原词，否则用原始文本
                if lemma_attr and lemma_attr.isdigit():
                    word = lemma_dict.get(int(lemma_attr), w.get_text().lower())
                else:
                    raw_word = w.get_text().lower()
                    word = lemmas.get(raw_word, raw_word)

                # 若非停用词则添加到 words 列表
                if word not in latin_stopwords:
                    words.append(word)

            # 返回处理后的文本字符串
            return ' '.join(words)

    # 文件不存在时，输出错误信息
    except FileNotFoundError:
        print(f"XML 文件未找到: {xml_file}")
        return ""


In [16]:
# 处理目录中的文件，保存结果
# 读取原始文件并处理，生成经过词形还原和停用词去除的文本文件
def process_directory(input_dir, output_dir):
    input_dir = os.path.join(BASE_DIR, input_dir)
    output_dir = os.path.join(BASE_DIR, output_dir)

    # 如果输出目录不存在，则创建
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 获取所有 XML 文件
    xml_files = [f for f in os.listdir(input_dir) if f.endswith('.xml')]

    for xml_file in tqdm.tqdm(xml_files, desc="Processing files"):
        input_path = os.path.join(input_dir, xml_file)
        output_path = os.path.join(output_dir, xml_file.replace('.xml', '.txt'))
        
        # 如果文件已处理则跳过
        if os.path.exists(output_path):
            print(f"文件已存在，跳过处理: {output_path}")
            continue
        
        try:
            # 调用预处理函数处理文本
            processed_text = preprocess_text(input_path)
            
            # 写入处理结果到输出文件
            with open(output_path, 'w') as out_file:
                out_file.write(processed_text)
            
            print(f"成功处理文件: {xml_file}")
        
        except Exception as e:
            print(f"处理文件 {xml_file} 时出错: {e}")


In [17]:
# 运行数据预处理
process_directory('data/raw', 'data/processed')

Processing files:   1%|          | 7/615 [00:00<00:14, 43.23it/s]

文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v170_048.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v170_060.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v170_074.txt
成功处理文件: h180_134.xml
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v180_135.txt
成功处理文件: h170_075.xml
成功处理文件: h170_061.xml
成功处理文件: h170_049.xml


Processing files:   5%|▍         | 28/615 [00:00<00:07, 77.44it/s]

成功处理文件: h190_281.xml
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v160_024_1.txt
成功处理文件: h190_256.xml
成功处理文件: h170_101.xml
成功处理文件: h170_115.xml
成功处理文件: h190_242.xml
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v190_243.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v170_114.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v170_100.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v190_257.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v190_280.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v190_281.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v190_256.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v170_101.txt
文件已存在，跳过处理: /Users/jessie/Doc

Processing files:   6%|▌         | 37/615 [00:00<00:10, 57.65it/s]

...

成功处理文件: h190_265.xml
成功处理文件: h190_271.xml
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v190_270.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v190_264.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v190_258.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v190_259.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v190_265.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v190_271.txt
成功处理文件: h190_270.xml
成功处理文件: h190_264.xml
成功处理文件: h190_258.xml
成功处理文件: h170_053.xml


Processing files:  96%|█████████▋| 593/615 [00:10<00:00, 60.90it/s]

成功处理文件: h170_047.xml
成功处理文件: h170_090.xml
成功处理文件: h170_084.xml
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v170_085.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v170_091.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v170_046.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v170_052.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v170_044.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v170_050.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v170_078.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v170_087.txt
成功处理文件: h180_138.xml
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v170_093.txt
成功处理文件: h170_079_b.xml
文件已存在，跳过处理: /Users/jessie/Doc

Processing files: 100%|██████████| 615/615 [00:10<00:00, 57.14it/s]

成功处理文件: h190_267.xml
成功处理文件: h190_273.xml
成功处理文件: h170_118.xml
成功处理文件: h170_044.xml
成功处理文件: h170_050.xml
成功处理文件: h170_078.xml
成功处理文件: h170_087.xml
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v180_138.txt
成功处理文件: h170_093.xml
成功处理文件: h180_139.xml
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v170_092.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v170_086.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v170_079.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v170_051.txt
文件已存在，跳过处理: /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/data/processed/v170_045.txt
